<a href="https://colab.research.google.com/github/c-larson/clean-test-course/blob/main/Praxa_Lesson_4_Complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lesson 3: Vector Database


##Getting Started
If you're new to Google Colab, download and review the [Getting Started with Colab](https://uploads.quantic.edu/assets/3330de11b37d7279540ad20e91425d070ced3ef92d7a5a296da55da40ca8d567/original/3330de11b37d7279540ad20e91425d070ced3ef92d7a5a296da55da40ca8d567.pdf) guide.

Your code and data will run in the `/content` directory.

The first task is to install libraries. Run the code in the following cell by clicking the play button on its left. Note that all commands at the shell prompt, such as `pip` below, should be preceded with a bang `!`. This will take a few minutes; while you're waiting you can get your OpenRouter API key (see the cell following this one for instructions).

If Colab asks you to restart the run time at the end of the library installations, go ahead and do so. We recommend rerunning this first cell, although theoretically you shouldn't have to. If you get dependency errors, please notify Quantic at techsupport+msse@quantic.edu.

In [14]:
!pip install langchain==0.3.25 langchain-community==0.3.24 pypdf==5.5.0 langchain-text-splitters==0.3.8 sentence_transformers==4.1.0 langchain-chroma==0.2.4 langchain-huggingface==0.2.0 openai==1.82.0 streamlit==1.45.1

You'll also need an API key from OpenRouter. Visit their [home page](https://openrouter.ai), select "Sign in" in the upper-right, then "Sign up" on the bottom of the dialog. The next dialog will have you create your account (we recommend using your Google account to sign in).

Once you've created an account, go to the hamburger menu in the upper right and select "Keys." Select "Create API Key" and follow the instructions.

Once you have your API key, enter it below and run the code in the cell.

In [31]:
import os
os.environ["OPENROUTER_API_KEY"] = "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"

##Loading Context Documents
The first step in building the vector database is to load the context documents. Load them into a variable named `context_data`.

In [32]:
import gdown
!mkdir context_data
gdown.download("https://quanticedu.github.io/praxa/Longest Running Shows on Broadway 2025.pdf", "./context_data/Longest Running Shows on Broadway.pdf", quiet=True)
gdown.download("https://quanticedu.github.io/praxa/Every play and musical coming to the West End in 2025.pdf", "./context_data/Every play and musical coming to the West End in 2025.pdf", quiet=True)
from langchain_community.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("./context_data")
context_data = loader.load()

mkdir: cannot create directory ‘context_data’: File exists


Now let's verify that the documents loaded by printing the content of each page. Scroll to the end of a line to see what metadata the document loader includes.

In [33]:
for page in context_data:
  print(page)

page_content='Every play and musical coming to the West End in 2025 
Here’s what’s beginning previews next year! 
 
As Christmas gets into full flow, now’s the perfect time to get excited for what’s cooking as 
we head towards 2025! Here’s plans for next year, based on the show that will begin 
previews at West End venues from January. 
Inside No 9 – Stage/Fright 
The ingenious minds of Steve Pemberton and Reece Shearsmith has resulted in many 
things, including nine seasons of the fantastic BBC series Inside No 9. The duo now bring 
the show to the stage, with a variety of familiar characters though now in a brand-new tale. 
Wyndham’s Theatre, from 18 January 
Elektra 
Oscar winner Brie Larson will be making her West End debut in the title role of Elektra. 
Directed by Daniel Fish, who brought us the hit Oklahoma! recently, the show features a 
new translation by Anne Carson and explores themes of grief, survival, and vengeance as 
Elektra is haunted by her father’s assassination. Duk

##Chunking
Now it's time to split the documents into chunks that will work with the LLM's context window. Store them in a variable named `chunks`.

In [34]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)
chunks = text_splitter.split_documents(context_data)

Verify it worked by exploring how the documents were chunked.

In [35]:
print(f"Total Document Chunks: {len(chunks)}")

for num, chunk in enumerate(chunks):
  print("------")
  print(f"Chunk {num}:")
  print(f"Length: {len(chunk.page_content)}")
  print(f"Metadata: {chunk.metadata}")
  print(f"Content: {chunk.page_content}")

Total Document Chunks: 17
------
Chunk 0:
Length: 931
Metadata: {'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-06-17T09:15:37-04:00', 'author': 'John Riehl', 'moddate': '2025-06-17T09:15:37-04:00', 'source': 'context_data/Every play and musical coming to the West End in 2025.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}
Content: Every play and musical coming to the West End in 2025 
Here’s what’s beginning previews next year! 
 
As Christmas gets into full flow, now’s the perfect time to get excited for what’s cooking as 
we head towards 2025! Here’s plans for next year, based on the show that will begin 
previews at West End venues from January. 
Inside No 9 – Stage/Fright 
The ingenious minds of Steve Pemberton and Reece Shearsmith has resulted in many 
things, including nine seasons of the fantastic BBC series Inside No 9. The duo now bring 
the show to the stage, with a variety of familiar characters t

##Embedding

Now it's time to set up the embedding function. Assign it to a variable named `embeddings_model`.

In [36]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

Make sure your model works by finding the embedding for a test sentence.

In [37]:
embedding = embeddings_model.embed_query("This is a test sentence.")
print(f"Embedding length: {len(embedding)}")
embedding = embeddings_model.embed_query("This is a longer test sentence.")
print(f"Embedding length: {len(embedding)}")

Embedding length: 384
Embedding length: 384


##Persisting

Now it's time for the vector store. Assign it the name `chromadb`.

In [38]:
from langchain_chroma import Chroma
chromadb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings_model,
    persist_directory="./chromadb"
)

Now test it by executing a similarity search.

In [39]:
retrieved_chunks = chromadb.similarity_search("A play written by Ryan Calais Cameron.")
print(f"Query retrieved {len(retrieved_chunks)} chunks.")
for chunk in retrieved_chunks:
  print(f"Chunk content: {chunk.page_content}")
  print(f"Chunk metadata: {chunk.metadata}")
  print("-----")

Query retrieved 4 chunks.
Chunk content: Retrograde 
Ryan Calais Cameron is one of the most exciting playwrights of the decade, so it’s brilliant 
seeing his Kiln Theatre three-hander making its way into the West End for a new spell. The 
piece focuses on a pivotal moment in Sidney Poitier’s early career as he faces a critical 
decision about signing a career-changing Hollywood contract. Apollo Theatre, from 8 
March 2025  
The Great Gatsby  
Good news, old sports! Broadway’s The Great Gatsby is heading to London for the summer. 
It is based on F Scott Fitzgerald’s 1925 novel about a self-made millionaire and his quest for 
the American Dream (in the arms of the married woman living across the bay). London 
Coliseum, from 11 April to 7 September 2025 
The Comedy About Spies 
The Mischief gang are back and up to their old tricks! This brand new show from the “Goes 
Wrong” pioneers looked fantastic when it appeared at the Royal Variety earlier this month,
Chunk metadata: {'moddate': '202

#Lesson 4: LangChain and Language Models

###Getting the LLM
OpenRouter offers free access to many LLMs: https://openrouter.ai/models?max_price=0

It is important to note that the free models have low rate limits (50 requests per day total), which is great for prototyping but usually not suitable for production use.

Since OpenRouter provides an API that is compatible with OpenAI, you can seamlessly integrate it into your LLM application by using the ChatOpenAI class from LangChain.

In [40]:
from langchain_community.chat_models import ChatOpenAI
from typing import Optional

class ChatOpenRouter(ChatOpenAI):
    openai_api_base: str
    openai_api_key: str
    model_name: str

    def __init__(self,
                 model_name: str,
                 openai_api_key: Optional[str] = None,
                 openai_api_base: str = "https://openrouter.ai/api/v1",
                 **kwargs):
        openai_api_key = openai_api_key or os.getenv('OPENROUTER_API_KEY')
        super().__init__(openai_api_base=openai_api_base,
                         openai_api_key=openai_api_key,
                         model_name=model_name, **kwargs)

llm = ChatOpenRouter(
    model_name="google/gemma-3-27b-it:free",
    max_tokens=512,
    temperature=0
)

Let's invoke the LLM with a few prompts it should be able to handle. Take note of the answers, which are based solely on the model's training data.

In [41]:
from langchain_core.messages import SystemMessage, HumanMessage
response = llm.invoke(
    [SystemMessage("You are a helpful assistant."),
     HumanMessage("What are some plays by Tawfiq al-Hakim?")])
print(response.content)
print("----------")
response = llm.invoke(
    [SystemMessage("You are a helpful assistant."),
     HumanMessage("What is Ryan Calais Camerons's most recent play?")])
print(response.content)
print("----------")
response = llm.invoke(
    [SystemMessage("You are a helpful assistant."),
     HumanMessage("What Broadway shows have more than 10,000 performances?")])
print(response.content)

Okay, here are some notable plays by the Egyptian playwright Tawfiq al-Hakim, categorized a bit for clarity, along with a little about each. He was *extremely* prolific, so this isn't exhaustive, but covers many of his most famous and important works.

**Early "Intellectual" or "Idea" Plays (Often philosophical and challenging traditional norms - 1930s-1950s)**

These are the plays that really established him as a major force in modern Arabic theatre. They often lack traditional plot structure and focus on debating ideas.

*   **_The Return of the Spirit_ (ʿAwdat al-Rūḥ, 1933):**  Considered his masterpiece and a cornerstone of modern Arabic drama. It's a symbolic play about a man who feels alienated from modern life and seeks a way to reconnect with his ancestral spirit and find meaning. It's very philosophical and explores themes of identity, tradition, and modernity.
*   **_Fate and the Donkey_ (Al-Maqdar wa al-Himār, 1936):** A satirical and philosophical play. It centers on a man 

###Setting up a Chat Prompt Template
We'll now build a simple prompt template to make our interface with the LLM a bit more generic.

In [43]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant."),
    ("user", "What is {playwright}'s most recent play?")
])

Let's test it out!

In [44]:
print(prompt_template.invoke({"playwright": "Ryan Calais Cameron"}))

response = llm.invoke(prompt_template.invoke({"playwright": "Ryan Calais Cameron"}))

print(response.content)

messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content="What is Ryan Calais Cameron's most recent play?", additional_kwargs={}, response_metadata={})]


Ryan Calais Cameron's most recent play is **"Punks Beat Eggshells"**. 

It premiered at the Royal Court Theatre in London in February 2024, and has received *very* positive reviews. It's a play about queer Black joy, resilience, and chosen family.

You can find more information about it here: [https://royalcourttheatre.com/whats-on/punks-beat-eggshells](https://royalcourttheatre.com/whats-on/punks-beat-eggshells)






## LangChain Expression Language (LCEL)
The "Chain" in "LangChain" refers to the ability to chain several actions into one invocation. This replaces your nested calls to `llm.invoke()`, and `chat_prompt.invoke()`. Try to build a chain for what you have here.

In [45]:
chain = prompt_template | llm
response = chain.invoke(
    {"playwright" : "Ryan Calais Cameron"})
print(response.content)



Ryan Calais Cameron's most recent play is **"Punks Beat Eggshells"**. 

It premiered at the Royal Court Theatre in London in February 2024, and has received *very* positive reviews. It's a play about queer Black joy, resilience, and chosen family.

You can find more information about it here: [https://royalcourttheatre.com/whats-on/punks-beat-eggshells](https://royalcourttheatre.com/whats-on/punks-beat-eggshells)






#Lesson 5: RAG Using LangChain

##Build a Prompt Template
We'll start with a prompt template that combines the context and original question and provides instructions to the model on how to use both.

In [46]:
prompt_template = ChatPromptTemplate([
    ("system", """You are an assistant
        providing answers to questions
        about the theater. In addition to
        your training data, you are to
        use the additional context
        provided below to provide
        up-to-date information."""),
    ("user", """Question:
        {question}\nContext:
        {context}""")])

To get the context, we'll use a *retriever*. It takes a string as the input query and returns a `list` of `Document` objects.

In [47]:
retriever = chromadb.as_retriever()

Run it to see what it outputs.

In [48]:
docs = retriever.invoke("What is Ryan Calais Cameron's most recent play?")
print(f"Found {len(docs)} documents:")

for doc in docs:
  print("------")
  print(doc)

Found 4 documents:
------
page_content='Harold Pinter Theatre, from 26 April to 2 August. 
  
The Deep Blue Sea 
Lindsay Posner’s production, first seen in Bath last year, features Tamsin Greig as Hester 
Collyer and Finbar Lynch as Miller. Terence Rattigan’s play delves into themes of obsession 
and the destructive power of love, and received a fantastic five-star write-up from 
WhatsOnStage earlier this year. Theatre Royal Haymarket, from 7 May to 21 June 2025' metadata={'producer': 'Microsoft® Word for Microsoft 365', 'page_label': '3', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-06-17T09:15:37-04:00', 'total_pages': 4, 'page': 2, 'author': 'John Riehl', 'source': 'context_data/Every play and musical coming to the West End in 2025.pdf', 'moddate': '2025-06-17T09:15:37-04:00'}
------
page_content='Harold Pinter Theatre, from 26 April to 2 August. 
  
The Deep Blue Sea 
Lindsay Posner’s production, first seen in Bath last year, features Tamsin Greig as Hester

The final form we're going for is `chain.invoke(user_question)`. We'll need the `user_question` for two things in this prompt: the question itself and finding the context from the vector database. Doing multiple things to one input is the job of a `RunnableParallel`. Let's create one that does that.

In [60]:
from langchain.schema.runnable import \
    RunnablePassthrough, RunnableParallel
question_and_docs = RunnableParallel(
    { "question": RunnablePassthrough(),
      "context_docs": retriever }
)

Let's see what that looks like.

In [61]:
question_and_docs.invoke("What is Ryan Calais Cameron's most recent play?")

{'question': "What is Ryan Calais Cameron's most recent play?",
 'context_docs': [Document(id='45f59dc9-13c5-42ea-bd87-2ea6490dd6f1', metadata={'total_pages': 4, 'creator': 'Microsoft® Word for Microsoft 365', 'page': 2, 'source': 'context_data/Every play and musical coming to the West End in 2025.pdf', 'moddate': '2025-06-17T09:15:37-04:00', 'producer': 'Microsoft® Word for Microsoft 365', 'author': 'John Riehl', 'creationdate': '2025-06-17T09:15:37-04:00', 'page_label': '3'}, page_content='Harold Pinter Theatre, from 26 April to 2 August. \n  \nThe Deep Blue Sea \nLindsay Posner’s production, first seen in Bath last year, features Tamsin Greig as Hester \nCollyer and Finbar Lynch as Miller. Terence Rattigan’s play delves into themes of obsession \nand the destructive power of love, and received a fantastic five-star write-up from \nWhatsOnStage earlier this year. Theatre Royal Haymarket, from 7 May to 21 June 2025'),
  Document(id='6f43e20a-7422-4405-98c2-751730444159', metadata={'cr

`RunnablePassthrough` has a static method named `assign()`, which adds keys to a dictionary by applying a function to that dictionary. Here's a simple example.

In [62]:
my_dict = {
    "question": "How much wood would a woodchuck chuck if a woodchuck could chuck wood?",
    "answer": "All the wood that a woodchuck could chuck if a woodchuck could chuck wood."
}

add_length = RunnablePassthrough.assign(length=len)
print(type(add_length))
add_length.invoke(my_dict)

<class 'langchain_core.runnables.passthrough.RunnableAssign'>


{'question': 'How much wood would a woodchuck chuck if a woodchuck could chuck wood?',
 'answer': 'All the wood that a woodchuck could chuck if a woodchuck could chuck wood.',
 'length': 2}

To use the context docs in a prompt, we're going to need to convert them to a string. We'll use a `RunnablePassthrough` to assign that string to the `context` key the prompt needs. Note that the `question` attribute from `context_docs_and_question` gets passed through.

In [65]:
def make_context_string(dict_with_docs):
    return "\n\n".join(doc.page_content for doc in dict_with_docs["context_docs"])

context = RunnablePassthrough.assign(
    context=make_context_string
)



Let's see how all this works with our prompt.

In [66]:
complete_prompt_chain = question_and_docs | context | prompt_template
complete_prompt_chain.invoke("What is Ryan Calais Cameron's most recent play?")

ChatPromptValue(messages=[SystemMessage(content='You are an assistant\n        providing answers to questions\n        about the theater. In addition to\n        your training data, you are to\n        use the additional context\n        provided below to provide\n        up-to-date information.', additional_kwargs={}, response_metadata={}), HumanMessage(content="Question: \n        What is Ryan Calais Cameron's most recent play?\nContext: \n        Harold Pinter Theatre, from 26 April to 2 August. \n  \nThe Deep Blue Sea \nLindsay Posner’s production, first seen in Bath last year, features Tamsin Greig as Hester \nCollyer and Finbar Lynch as Miller. Terence Rattigan’s play delves into themes of obsession \nand the destructive power of love, and received a fantastic five-star write-up from \nWhatsOnStage earlier this year. Theatre Royal Haymarket, from 7 May to 21 June 2025\n\nHarold Pinter Theatre, from 26 April to 2 August. \n  \nThe Deep Blue Sea \nLindsay Posner’s production, first

Now we'll build the final chain for our app.

In [67]:
chain = (question_and_docs
    | context
    | prompt_template
    | llm
)

And run it to see what results we get. Here we should see that "Retrograde" is the response, even though it occurred after the training cutoff of the model.

If you don't see "Retrograde", try starting a new Colab runtime and running the code in the notebook again. This resets the model and clears cached responses.

In [68]:
result = chain.invoke("What is Ryan Calais Cameron's most recent play?")
print(result.content)


According to the information provided, Ryan Calais Cameron's most recent play is **Retrograde**, which is playing at the Apollo Theatre from 8 March 2025.


Now we'll build a chain that passes the source citations, which were in the metadata field of the `list` of `Document` objects returned from the retriever. We'll use `RunnableParallel` to pass the `list` to the end of the chain while also passing it to a chain that builds the prompt and invokes the model.

In [69]:
answer_chain = (context
    | prompt_template
    | llm)
chain_with_sources = \
    question_and_docs.assign(
        answer=answer_chain)

Now run it to see what we got. When we asked this question without context, the model listed dated figures (e.g., 10,737 shows for "Chicago"). The model read the context data and formed a response that answered the question directly.

In [70]:
result = chain_with_sources.invoke("What Broadway shows have more than 10,000 performances?")
print("The docs used in this answer:")
print("\n".join(doc.metadata.__repr__() for doc in result["context_docs"]))
print("----------------------------")
print("\nThe answer:")
print(result["answer"].content)

The docs used in this answer:
{'page': 0, 'total_pages': 6, 'page_label': '1', 'author': 'John Riehl', 'creationdate': '2025-06-17T09:20:44-04:00', 'source': 'context_data/Longest Running Shows on Broadway.pdf', 'creator': 'Microsoft® Word for Microsoft 365', 'producer': 'Microsoft® Word for Microsoft 365', 'moddate': '2025-06-17T09:20:44-04:00'}
{'page': 0, 'source': 'context_data/Longest Running Shows on Broadway.pdf', 'moddate': '2025-06-17T09:20:44-04:00', 'page_label': '1', 'total_pages': 6, 'author': 'John Riehl', 'producer': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-06-17T09:20:44-04:00', 'creator': 'Microsoft® Word for Microsoft 365'}
{'moddate': '2025-06-17T09:20:44-04:00', 'creationdate': '2025-06-17T09:20:44-04:00', 'page_label': '1', 'creator': 'Microsoft® Word for Microsoft 365', 'total_pages': 6, 'page': 0, 'source': 'context_data/Longest Running Shows on Broadway.pdf', 'author': 'John Riehl', 'producer': 'Microsoft® Word for Microsoft 365'}
{'source': 'c

#Lesson 6: User Interface
While not directly related to LLMs or AI in general, user interfaces are essential to making an app approachable. We'll use Streamlit to build a basic front end for our app.
##Getting Started
First we need to install an npm package that will allow us to expose the Colab runtime to IP traffic.

In [ ]:
!pip install streamlit
!npm install localtunnel


Now we'll create a simple "Hello World" app. This illustrates how simple Streamlit is to use.

In [ ]:
%%writefile app.py
import streamlit as st
st.write("Hello World!")

Now we need to run the app and view it in a browser. There are a few steps to this:
* Start the Streamlit server using the *app.py* script.
* Set up a local tunnel to get a URL that connects to the Colab runtime.
* Get the public IP of the Colab runtime to gain access to the localtunnel-created URL.

We'll do this all on one command line. The command will display the public IP of the Colab runtime then a link to the Streamlit server. When you click the link you'll be asked for the tunnel password, which is the Colab runtime's public IP.

Note that this cell will continue running the server until you manually stop it. No other cells in the notebook can run while this cell is running. Stop the cell by selecting the stop button to its left.

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.58.10.35
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) 

##Building the Backend
Up to this point we've been running the Python instructions in interactive mode in the Colab notebook. For our app to work as a backend, we need to make the code available in a module that the front end can import. Let's go back through our code and copy the needed elements into a single Python file.

In [ ]:
%%writefile backend.py
import os
from langchain.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.schema.runnable import RunnablePassthrough, RunnableParallel
from langchain_community.chat_models import ChatOpenAI
from typing import Optional

# 1. Set ChatPromptTemplate for RAG
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant providing answers to questions about the theater. "
               "In addition to your training data, use the additional context provided below to provide up-to-date information."),
    ("user", "Question: {question}\nContext: {context}\nAnswer:")
])

# 2. Context Retriever
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = Chroma(persist_directory="./chromadb",
                  embedding_function=embedding_function)
retriever = vectordb.as_retriever()

#3. LCEL Chain Setup
question_and_docs = RunnableParallel(
    {"context_docs": retriever, "question": RunnablePassthrough()}
)

def make_context_string(to_convert):
    # Take the page_content attribute of each Document object
    # and join them into one string, separated by two newlines.
    return "\n\n".join(doc.page_content for doc in to_convert["context_docs"])

context = RunnablePassthrough.assign(context=make_context_string)

# 4. LLM
class ChatOpenRouter(ChatOpenAI):
    openai_api_base: str
    openai_api_key: str
    model_name: str

    def __init__(self,
                 model_name: str,
                 openai_api_key: Optional[str] = None,
                 openai_api_base: str = "https://openrouter.ai/api/v1",
                 **kwargs):
        openai_api_key = openai_api_key or os.getenv('OPENROUTER_API_KEY')
        super().__init__(openai_api_base=openai_api_base,
                         openai_api_key=openai_api_key,
                         model_name=model_name, **kwargs)

llm = ChatOpenRouter(
    model_name="google/gemma-3-27b-it:free",
    max_tokens=512,
    temperature=0
)

# 5. Build answering Chain
answer_chain = context | prompt | llm

chain_with_sources = question_and_docs.assign(answer=answer_chain)


# 6. Final method to invoke the chain and get answer and sources
def answer_and_sources(question):
    result = chain_with_sources.invoke(question)
    response_text = result["answer"].content
    #answer_index = response_text.rfind("Answer:")
    #answer_text = response_text[answer_index + len("Answer:"):].strip()
    sources = "\n\n".join(f"{doc.metadata['source']}, page {doc.metadata['page']}" for doc in result["context_docs"])
    #return {"answer": answer_text,
    return {"answer": response_text,
            "sources": sources
            }

Now test the back end manually to make sure it works.

In [ ]:
import backend, importlib
importlib.reload(backend)
print(backend.answer_and_sources("What is Ryan Calais Cameron's most recent play?"))

##Building the Interface
Now let's use Streamlit's example chat app to build the interface for Praxa.

Now we run the whole app.

In [ ]:
!streamlit run praxa.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com